# Submitting calculations
#### Create a structure, kpoints, and input parameters and submit a Quantum ESPRESSO (PW) calculation
Time: 3 mins

In [7]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm.utils import load_node
from aiida.work.run import run
from aiida.orm.code import Code
from aiida.orm.data.base import Bool, Str
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.structure import StructureData
from ase.lattice.spacegroup import crystal

import numpy as np

#### Create a diamond cubic crystal structure

In [8]:
# Define a structure, employing the Atomic Structure Environment library
alat = 3.568
ase_definition = crystal('C', [(0,0,0)], spacegroup=227, cellpar=[alat, alat, alat, 90, 90, 90], primitive_cell=True)*2
structure = StructureData(ase=ase_definition)
structure.store()

<StructureData: uuid: 5da63df5-7993-4a69-b42f-cbb94861414e (pk: 5214)>

#### Create the _k_-points mesh

In [9]:
# Define the k-points mesh
kpoints = KpointsData()
kpoints_mesh = [1, 1, 1]
kpoints.set_kpoints_mesh(kpoints_mesh)
kpoints.store()

<KpointsData: uuid: ca07ac8f-5646-41d0-ab52-2ef7d83e9b47 (pk: 5215)>

#### List existing pseudopotential families

In [10]:
!verdi data upf listfamilies

  class AiidaObjectManager(m.Manager):

* GBRV_pbe [33 pseudos]
* GBRV_lda [33 pseudos]
* GBRV_pbesol [33 pseudos]
* SSSP [85 pseudos]


#### Define the calculation input parameters

In [11]:
# Define the pseudo potential family and input parameters for pw.x
pseudo_family = 'SSSP'
parameters = {
    'CONTROL': {
        "calculation": "scf",
        'tstress': True,
    },
    'SYSTEM': {
        'ecutwfc': 40.,
        'ecutrho': 320.,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

#### See the available codes and select one of them

In [12]:
!verdi code list

  class AiidaObjectManager(m.Manager):

# List of configured codes:
# (use 'verdi code show CODEID' to see the details)
* pk 4681 - pw-5.1@localhost


#### Get the Code instance from the name

In [13]:
codename = 'pw-5.1@localhost'
code = Code.get_from_string(codename)

#### Create a new calculation from the code and set the input parameters

In [14]:
calc = code.new_calc()
calc.use_structure(structure)
calc.use_pseudos_from_family(pseudo_family)
calc.use_parameters(ParameterData(dict=parameters))
calc.use_kpoints(kpoints)
calc.set_resources({"num_machines": 1})
calc.set_max_wallclock_seconds(30*60)

#### Store the calculation and submit it to the daemon

In [15]:
calc.store_all()
calc.submit()
print "Submitted a {} with pk<{}>".format(calc.__class__.__name__, calc.pk)

Submitted a PwCalculation with pk<5217>


#### Check the status of the calculation

In [16]:
!verdi calculation list

  class AiidaObjectManager(m.Manager):

# Last daemon state_updater check: 0h:09m:22s ago (at 09:53:12 on 2017-04-24)
  PK  State     Creation      Sched. state  Computer    Type
----  --------  ----------  --------------  ----------  ------------------
5217  TOSUBMIT  3s ago                      localhost   quantumespresso.pw

Total results: 1



#### Monitor its status until completion

In [17]:
import time
from IPython.display import clear_output

def check_calculation_status(calc):
    while not calc.has_finished():
        clear_output()
        !verdi calculation list
        time.sleep(2)

    clear_output()
    !verdi calculation list $calc.pk

In [18]:
check_calculation_status(calc)

  class AiidaObjectManager(m.Manager):

# Last daemon state_updater check: 0h:00m:00s ago (at 10:03:25 on 2017-04-24)
  PK  State     Creation    Sched. state    Computer    Type
----  --------  ----------  --------------  ----------  ------------------
5217  FINISHED  55s ago     DONE            localhost   quantumespresso.pw

Total results: 1



#### Check that the calculation finished successfully

In [19]:
calc.has_finished_ok()

True

#### We can easily show all the attached outputs

In [20]:
calc.get_outputs_dict()

{u'output_array': <ArrayData: uuid: 7b4cf775-e8e0-479d-84f8-f8f9cd758d5c (pk: 5222)>,
 u'output_array_5222': <ArrayData: uuid: 7b4cf775-e8e0-479d-84f8-f8f9cd758d5c (pk: 5222)>,
 u'output_kpoints': <KpointsData: uuid: 93cff57e-0ff0-4d78-a000-b0337cefcf75 (pk: 5220)>,
 u'output_kpoints_5220': <KpointsData: uuid: 93cff57e-0ff0-4d78-a000-b0337cefcf75 (pk: 5220)>,
 u'output_parameters': <ParameterData: uuid: f13ae77f-5cd5-4643-bc66-3bbee0563492 (pk: 5221)>,
 u'output_parameters_5221': <ParameterData: uuid: f13ae77f-5cd5-4643-bc66-3bbee0563492 (pk: 5221)>,
 u'remote_folder': <RemoteData: uuid: c417697a-eee9-4d31-afc9-79ca05a2b59e (pk: 5218)>,
 u'remote_folder_5218': <RemoteData: uuid: c417697a-eee9-4d31-afc9-79ca05a2b59e (pk: 5218)>,
 u'retrieved': <FolderData: uuid: 9913d36c-bf77-48c9-a1fe-71da02f495f4 (pk: 5219)>,
 u'retrieved_5219': <FolderData: uuid: 9913d36c-bf77-48c9-a1fe-71da02f495f4 (pk: 5219)>}

#### There are convenient methods to directly access the results

In [21]:
print "Total energy = {} {}".format(calc.res.energy, calc.res.energy_units)

Total energy = -2471.82841453 eV
